# New Jersey Education ETL
---
## Sections:
1. Scraping graduation data from The State of New Jersey Department of Education (URL: <https://www.state.nj.us/education/data/grate/>).
2. Scraping population and median household income data for each district from the United States Census Bureau.
3. Merging the data scraped from parts 1 and 2 and loading into MongoDB.


In [1]:
import pandas as pd
import datetime
import pymongo
import json
from bson import Binary, Code, json_util, ObjectId
from bson.json_util import dumps, RELAXED_JSON_OPTIONS
import pygeoj
import os
import itertools 
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
from us import states

# Create a Mongo Connection

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.education_data

# Part 1: Scraping New Jersey Education Data
This section includes:
1. Scraping the graduation rates for each school over the period of 2014 to 2018
2. Scraping the distric budget over the period of 2014 to 2018

In [3]:
# Section 1 - Scraping for graduation rates

# file location
rootdir = './data/4YearGraduation'

# years to scrape data from
districts_list = []
years = ['2018','2015','2014','2016','2017']

# loop through the directory to scrape all files
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        data = os.path.join(subdir, file)
        districts_list.append(data)

districts_list.pop(0)
grad_list=[]
#  scrape for graduation data
for (file, year) in zip(districts_list, years):
    data = file
    # defined variables labels
    grad_label = "grad_"+ year
    df_label = grad_label +"_df"
    
    # Read data & add to dataframe
    grad_label = pd.read_csv(data)
    df_label = pd.DataFrame(grad_label)

    # Transform the scraped data
    # replace * with 0
    df_label['FOUR_YR_GRAD_RATE'] = df_label['FOUR_YR_GRAD_RATE'].str.replace('*','0')
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = df_label['FOUR_YR_ADJ_COHORT_COUNT'].str.replace('*','0')
    df_label['GRADUATED_COUNT'] = df_label['GRADUATED_COUNT'].str.replace('*','0')
    df_label['FOUR_YR_GRAD_RATE'] = df_label['FOUR_YR_GRAD_RATE'].str.replace('-','0')
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = df_label['FOUR_YR_ADJ_COHORT_COUNT'].str.replace('-','0')
    df_label['GRADUATED_COUNT'] = df_label['GRADUATED_COUNT'].str.replace('-','0')
    # set previous change to numeric (int)
    df_label['FOUR_YR_GRAD_RATE'] = pd.to_numeric(df_label['FOUR_YR_GRAD_RATE'])
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = pd.to_numeric(df_label['FOUR_YR_ADJ_COHORT_COUNT'])
    df_label['GRADUATED_COUNT'] = pd.to_numeric(df_label['GRADUATED_COUNT'])
    grad_list.append(df_label)

# Create a dataframe 
graduation_df = pd.concat(grad_list, axis=0, ignore_index=True, sort=False)

graduation_df = graduation_df.filter(['YEAR','COUNTY_NAME', 'DISTRICT_NAME','SCHOOL_NAME','SUBGROUP','FOUR_YR_ADJ_COHORT_COUNT','FOUR_YR_GRAD_RATE','GRADUATED_COUNT'])
# Cast year as an integer
graduation_df['YEAR']=graduation_df['YEAR'].astype(int)

graduation_df = graduation_df.sort_values(by=['YEAR'])

# Output to JN
graduation_df.head()

,YEAR,COUNTY_NAME,DISTRICT_NAME,SCHOOL_NAME,SUBGROUP,FOUR_YR_ADJ_COHORT_COUNT,FOUR_YR_GRAD_RATE,GRADUATED_COUNT
0,2014,ATLANTIC,ATLANTIC CITY,ATLANTIC CITY HIGH SCHOOL,White,113,82.3,93
5083,2014,MORRIS,MADISON BORO,Districtwide Total,Economically Disadvantaged,0,0.0,0
5082,2014,MORRIS,MADISON BORO,Districtwide Total,Limited English Proficiency,0,0.0,0
5081,2014,MORRIS,MADISON BORO,Districtwide Total,Two or More Races,0,0.0,0
5080,2014,MORRIS,MADISON BORO,Districtwide Total,Native Hawaiian,0,0.0,0


In [4]:
# Section 2 - Scraping for budget data

# files
rootdir = './data/district'

# lists defined
budget_list = []
years = ['2017_2018','2016_2017','2015_2016','2014_2015','2013_2014']

# for loop to get all files
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        data = os.path.join(subdir, file)
        budget_list.append(data)
        budget_list.sort()
        budget_list.reverse()
        

sub_budget_list=[]
#  add all 4 year graduation from 2013 to 2018 to mongod
for (file, year) in zip(budget_list, years):
    data = file
    # defined variables labels
    budget_label = "district_budget"+ year
    df_label = budget_label +"_df"    
    
    # Read data & add to dataframe
    grad_label = pd.read_csv(data)
    df_label = pd.DataFrame(grad_label)
    df_label = df_label.filter(['YEAR','COUNTY_NAME', 'DISTRICT_NAME','TOTAL'])
    df_label = df_label.rename(index=str, columns={"TOTAL": "DISTRICT_BUDGET"})
    sub_budget_list.append(df_label)

budget_df = pd.concat(sub_budget_list, axis=0, ignore_index=True)
budget_df['YEAR']=budget_df['YEAR'].astype(int)
budget_df = budget_df.sort_values(by=['YEAR'])

# Merge the dataframe with the budget data and the graduation data
education_df = pd.merge(budget_df,
                graduation_df,
                how='inner',
                on=['YEAR', 'COUNTY_NAME', 'DISTRICT_NAME'])

# Output to JN
education_df.head()

,YEAR,COUNTY_NAME,DISTRICT_NAME,DISTRICT_BUDGET,SCHOOL_NAME,SUBGROUP,FOUR_YR_ADJ_COHORT_COUNT,FOUR_YR_GRAD_RATE,GRADUATED_COUNT
0,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Native Hawaiian,0,0.0,0
1,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,American Indian,0,0.0,0
2,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Asian,0,0.0,0
3,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Hispanic,0,0.0,0
4,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Black,0,0.0,0


# Part 2: Scraping Census Data
This section includes:
1. Scraping for demographics and household income data.
2. Scraping for population data.

In [5]:
# Section 1 - Scraping for demographics and household income data for each district

census_data = './data/citylocs_geocodio.csv'
census_data = pd.read_csv(census_data)
demographics_df = pd.DataFrame(census_data)
demographics_df.columns = demographics_df.columns.str.replace(" ", "_")

# Drop columns not required
demographics_df = demographics_df.drop(columns=['Street', 'Country', 'State'])

# Extract all the counties to be filtered from original dataset for census
lstCounty = education_df["COUNTY_NAME"].str.strip().tolist()
lstUniqueCounty = pd.unique(lstCounty)

# Converting census data to match the 
demographics_df['County'] = demographics_df['County'].str.upper()
demographics_df['County'] = demographics_df['County'].str.replace('COUNTY','').str.strip()

# Filter only relevant data
demographics_filtered_df = demographics_df[demographics_df['County'].isin(lstUniqueCounty)]

# Output to JN window
demographics_filtered_df.head()

,DISTRICT_NAME,Latitude,Longitude,City,County,Zip,Number_of_households,Median_household_income,Median_age,Median_age_male,Median_age_female,"Population_by_minimum_level_of_education/Male:_12th_grade,_no_diploma",Population_by_minimum_level_of_education/Male:_High_school_graduate,Population_by_minimum_level_of_education/Female:_12th_grade_no_diploma,Population_by_minimum_level_of_education/Female:_High_school_graduate
0,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,43.6,0.000,0.210,0.000,0.193
1,WOODBURY CITY,39.8382,-75.1527,Woodbury,GLOUCESTER,8096,785.0,33205.0,43.0,41.7,44.3,0.015,0.450,0.000,0.368
2,WOODBRIDGE TWP,40.5576,-74.2846,Woodbridge,MIDDLESEX,7095,462.0,82870.0,41.9,38.7,43.2,0.033,0.496,0.000,0.427
3,WOOD-RIDGE BORO,40.8457,-74.0879,Wood Ridge,BERGEN,7075,250.0,0.0,40.0,39.9,43.0,0.000,0.356,0.000,0.407
4,WINSLOW TWP,39.6573,-74.8624,Winslow,CAMDEN,8095,487.0,91932.0,37.8,38.8,35.1,0.000,0.384,0.039,0.376


In [6]:
# Scrape population data
population_data = './data/census_pop.csv'
population_data = pd.read_csv(population_data)
population_df = pd.DataFrame(population_data)
population_df = population_df.rename(columns = {'Zipcode': 'Zip'})
population_df.head()

,Population,Zip
0,22121.0,1832
1,8295.0,1833
2,6675.0,1834
3,13527.0,1835
4,4547.0,1840


In [7]:
# Merge the population dataframe with the district census data
census_df = pd.merge(demographics_filtered_df,
                population_df,
                how='left',
                on='Zip')
# Output to JN
census_df.head()

,DISTRICT_NAME,Latitude,Longitude,City,County,Zip,Number_of_households,Median_household_income,Median_age,Median_age_male,Median_age_female,"Population_by_minimum_level_of_education/Male:_12th_grade,_no_diploma",Population_by_minimum_level_of_education/Male:_High_school_graduate,Population_by_minimum_level_of_education/Female:_12th_grade_no_diploma,Population_by_minimum_level_of_education/Female:_High_school_graduate,Population
0,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,43.6,0.000,0.210,0.000,0.193,9129.0
1,WOODBURY CITY,39.8382,-75.1527,Woodbury,GLOUCESTER,8096,785.0,33205.0,43.0,41.7,44.3,0.015,0.450,0.000,0.368,35116.0
2,WOODBRIDGE TWP,40.5576,-74.2846,Woodbridge,MIDDLESEX,7095,462.0,82870.0,41.9,38.7,43.2,0.033,0.496,0.000,0.427,20722.0
3,WOOD-RIDGE BORO,40.8457,-74.0879,Wood Ridge,BERGEN,7075,250.0,0.0,40.0,39.9,43.0,0.000,0.356,0.000,0.407,8071.0
4,WINSLOW TWP,39.6573,-74.8624,Winslow,CAMDEN,8095,487.0,91932.0,37.8,38.8,35.1,0.000,0.384,0.039,0.376,180.0


# Part 3 - Merging Data and Loading to MongoDB
This section includes:
1. Merging the final dataframes education_df and census_df from parts 1 and 2.
2. Loading the data into MongoDB.

In [8]:
# Merge the data frames from parts 1 and 2
final_df = census_df.merge(education_df, left_on=['County', 'DISTRICT_NAME'], right_on=['COUNTY_NAME', 'DISTRICT_NAME'], how='inner')

# Output to JN window
final_df.head()

,DISTRICT_NAME,Latitude,Longitude,City,County,Zip,Number_of_households,Median_household_income,Median_age,Median_age_male,...,Population_by_minimum_level_of_education/Female:_High_school_graduate,Population,YEAR,COUNTY_NAME,DISTRICT_BUDGET,SCHOOL_NAME,SUBGROUP,FOUR_YR_ADJ_COHORT_COUNT,FOUR_YR_GRAD_RATE,GRADUATED_COUNT
0,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Native Hawaiian,0,0.0,0
1,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,American Indian,0,0.0,0
2,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Asian,0,0.0,0
3,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Hispanic,0,0.0,0
4,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Black,0,0.0,0


In [9]:
# clean merged data set by removing all demographics with no data for a given subgroup
final_df = final_df[final_df.GRADUATED_COUNT !=0]

# convert the dataframe to a diciontary, which can be pushed to mongo
final_dict = final_df.to_dict('records')

# create mongo collection
collection = db.education_data
# drop collection if duplicate
db.collection.drop()
# add data to collection
collection.insert_many(final_dict)

# Store dataframe as csv as well
final_df.to_csv(r'./data/merge_data.csv')